In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
from processing import Process

conf = SparkConf()
# conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
# conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

procdata = Process()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/29 22:10:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
geralprouni = pd.DataFrame(pd.read_xml("pure_data/geralprouni.xml"))

quantidade_bolsas = geralprouni[['bolsa_integral_cotas', 'bolsa_integral_ampla', 'bolsa_parcial_cotas', 'bolsa_parcial_ampla']].sum(axis=1)
nota_bolsa = geralprouni[['nota_integral_ampla', 'nota_integral_cotas', 'nota_parcial_ampla', 'nota_parcial_cotas']].mean(axis=1)

prouni = pd.DataFrame({
    'mensalidade': geralprouni['mensalidade'], 
    'estado': geralprouni['uf_busca'], 
    'municipio': geralprouni['cidade_busca'].map(procdata.no_accents_lower),
    'curso': geralprouni['nome'], 
    'quantidade_bolsas': quantidade_bolsas,
    'nota_bolsa': nota_bolsa
})
#geralprouni.head(5)
#print(geralprouni['cidade_busca'].head())
print(prouni.__len__(), "rows")
prouni.sample(n=10)

41447 rows


,mensalidade,estado,municipio,curso,quantidade_bolsas,nota_bolsa
35291,257.19,MG,pocos de caldas,Gestão Ambiental,2.0,450.00
24967,319.00,BA,guanambi,Pedagogia,1.0,596.06
13960,648.36,DF,brasilia,Design de Interiores,2.0,608.70
24349,325.00,PA,itaituba,Administração,1.0,529.58
24981,319.00,DF,brasilia,Redes de Computadores,1.0,538.94
2472,1420.85,MA,sao luis,Engenharia Mecânica,4.0,577.11
26568,299.00,AL,sao miguel dos campos,Pedagogia,2.0,557.34
35219,259.00,TO,palmas,Embelezamento e Imagem Pessoal,4.0,535.74
26676,299.00,ES,vila velha,Pedagogia,1.0,691.80
24569,325.00,RJ,macae,Letras - Português e Espanhol,1.0,506.34


In [3]:
addprouni = pd.read_json("pure_data/prouni.json")

adicionalprouni = pd.DataFrame({
    'curso': addprouni['NOME_CURSO_BOLSA'],
    'estado': addprouni['SIGLA_UF_BENEFICIARIO_BOLSA'],
    'municipio': addprouni['MUNICIPIO_BENEFICIARIO_BOLSA'].map(procdata.no_accents_lower),
    'sexo': addprouni['SEXO_BENEFICIARIO_BOLSA'].map(lambda sexo: 'Masculino' if sexo == 'M' else 'Feminino'),
    'raca': addprouni['RACA_BENEFICIARIO_BOLSA'].map(lambda raca: 'Outro' if raca not in ['Parda', 'Branca', 'Preta'] else raca),
    'deficiente': addprouni['BENEFICIARIO_DEFICIENTE_FISICO'].map(lambda deficiente: 'Sim' if deficiente == 'S' else 'Não'),
    'idade': addprouni['DT_NASCIMENTO_BENEFICIARIO'].map(procdata.get_age_from_birthdate)
})
#addprouni.head(3)
print(addprouni.columns)
print(adicionalprouni.__len__(), "rows")
adicionalprouni.sample(n=10)

Index(['ANO_CONCESSAO_BOLSA', 'CODIGO_EMEC_IES_BOLSA', 'NOME_IES_BOLSA',
       'TIPO_BOLSA', 'MODALIDADE_ENSINO_BOLSA', 'NOME_CURSO_BOLSA',
       'NOME_TURNO_CURSO_BOLSA', 'CPF_BENEFICIARIO_BOLSA',
       'SEXO_BENEFICIARIO_BOLSA', 'RACA_BENEFICIARIO_BOLSA',
       'DT_NASCIMENTO_BENEFICIARIO', 'BENEFICIARIO_DEFICIENTE_FISICO',
       'REGIAO_BENEFICIARIO_BOLSA', 'SIGLA_UF_BENEFICIARIO_BOLSA',
       'MUNICIPIO_BENEFICIARIO_BOLSA'],
      dtype='object')
236636 rows


,curso,estado,municipio,sexo,raca,deficiente,idade
122494,Administração,MG,belo horizonte,Masculino,Preta,Não,39
69117,Gastronomia,DF,brasilia,Feminino,Preta,Não,29
177712,Engenharia Civil,DF,brasilia,Masculino,Parda,Não,22
219193,Fonoaudiologia,AC,rio branco,Masculino,Preta,Não,24
161255,Serviço Social,BA,formosa do rio preto,Masculino,Parda,Não,32
27181,Geografia,RS,santana do livramento,Masculino,Branca,Não,32
233860,Engenharia Civil,PB,joao pessoa,Masculino,Parda,Não,33
106418,Geografia,PR,palmital,Masculino,Branca,Não,22
170866,Direito,SP,sao paulo,Feminino,Branca,Não,23
17766,Química Industrial,SP,piracicaba,Feminino,Branca,Não,23


In [4]:
edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")

educacaobasica = pd.DataFrame({
    'estado': edbasica['SG_UF'],
    'municipio': edbasica['NO_MUNICIPIO'].map(procdata.no_accents_lower),
    'esgoto_inexistente': edbasica['IN_ESGOTO_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'energia_inexistente': edbasica['IN_ENERGIA_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'agua_inexistente': edbasica['IN_AGUA_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'acesso_internet': edbasica['IN_INTERNET'].map(procdata.get_label_from_floatbool),
    'faz_exame_selecao': edbasica['IN_EXAME_SELECAO'].map(procdata.get_label_from_floatbool),
    'especializada_deficientes': edbasica['IN_ESP'].map(procdata.get_label_from_floatbool),
    'ensino_tecnico': edbasica['IN_PROF_TEC'].map(procdata.get_label_from_floatbool)
})
#dfedbasica = spark.createDataFrame(geralprouni)
#edbasica.head(10)
print(edbasica[edbasica['SG_UF']=='MG'].sample(10))
print(educacaobasica.__len__(), "rows")
educacaobasica.sample(10)

/var/folders/90/4r9_jk_s59n2616g7y9snmn00000gn/T/ipykernel_29276/3938479507.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")


        NU_ANO_CENSO NO_REGIAO  CO_REGIAO         NO_UF SG_UF  CO_UF  \
108895          2021   Sudeste          3  Minas Gerais    MG     31   
123774          2021   Sudeste          3  Minas Gerais    MG     31   
116973          2021   Sudeste          3  Minas Gerais    MG     31   
127936          2021   Sudeste          3  Minas Gerais    MG     31   
121935          2021   Sudeste          3  Minas Gerais    MG     31   
119910          2021   Sudeste          3  Minas Gerais    MG     31   
126806          2021   Sudeste          3  Minas Gerais    MG     31   
118353          2021   Sudeste          3  Minas Gerais    MG     31   
121292          2021   Sudeste          3  Minas Gerais    MG     31   
109849          2021   Sudeste          3  Minas Gerais    MG     31   

          NO_MUNICIPIO  CO_MUNICIPIO                    NO_MESORREGIAO  \
108895           Betim       3106705   Metropolitana de Belo Horizonte   
123774        Rio Novo       3155405                      Z

,estado,municipio,esgoto_inexistente,energia_inexistente,agua_inexistente,acesso_internet,faz_exame_selecao,especializada_deficientes,ensino_tecnico
155242,SP,hortolandia,Não,Não,Não,Sim,Sim,Não,Não
109672,MG,buritis,Não,Não,Não,Não,Não,Não,Não
205709,RS,quarai,Não,Não,Não,Não,Não,Sim,Não
101885,BA,sao felix,Não,Não,Não,Sim,Não,Sim,Não
69826,PE,buique,Não,Não,Não,Sim,Não,Sim,Não
143643,RJ,rio de janeiro,Não,Não,Não,Sim,Não,Sim,Não
137662,RJ,nilopolis,Não,Não,Não,Não,Não,Não,Não
39615,MA,tutoia,Não,Não,Não,Não,Não,Sim,Não
97533,BA,paramirim,Não,Não,Não,Sim,Não,Sim,Não
76327,PE,recife,Não,Não,Não,Sim,Não,Sim,Não


In [5]:
# SPARK

# df = spark.createDataFrame(prouni)
# dfadicionalprouni = spark.createDataFrame(adicionalprouni.drop_duplicates(subset=['estado', 'curso']))
# dfeducacaobasica = spark.createDataFrame(educacaobasica.drop_duplicates(subset=['estado']))
# df_joinadicional = df.join(dfadicionalprouni, on=['estado', 'curso'], how='left')
# df_joinedbasica = df_joinadicional.join(dfeducacaobasica, on='estado', how='left')
# dfmaster = df_joinedbasica.dropna()

# PANDAS

df = prouni.copy()
dfadicionalprouni = adicionalprouni.drop_duplicates(subset=['estado', 'curso', 'municipio'])
dfeducacaobasica = educacaobasica.drop_duplicates(subset=['estado', 'municipio'])
df_joinadicional = df.merge(dfadicionalprouni, on=['estado', 'curso', 'municipio'], how='left')
df_joinedbasica = df_joinadicional.merge(dfeducacaobasica, on=['estado', 'municipio'], how='left')
dfmaster = df_joinedbasica.dropna()

In [6]:
# SPARK
# dfmaster.coalesce(1).write.csv('processed_data/prouni.csv')

# PANDAS
dfmaster.to_csv('processed_data/prouni-processado.csv')